In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from netCDF4 import Dataset
import cftime

# Read and filter unwanted data

In [2]:
def CYGdf(DDMfilename):
    filedslist=["sp_lat","sp_lon","sp_alt",'prn_code',"ddm_snr","fresnel_coeff",
           "sp_inc_angle","sp_rx_gain",'gps_eirp','gps_ant_gain_db_i',
           'gps_tx_power_db_w',"ddm_noise_floor","quality_flags",'quality_flags_2',"rx_to_sp_range",
           "tx_to_sp_range","add_range_to_sp",'track_id','sv_num']
    d=Dataset(DDMfilename,'r')
    dicts={key:np.array(d[key]).reshape(-1) for key in filedslist}
    df=DataFrame(dicts)
    datearr=cftime.num2pydate(d['ddm_timestamp_utc'][:],units=d['ddm_timestamp_utc'].units).data.repeat(4).reshape(-1)
    df['ddm_timestamp_utc']=datearr
    QF1=df['quality_flags'].values
    QF2=df['quality_flags_2'].values 
    #sp_over_land=0x00000400
    #s_band_powered_up=0x00000002
    #large_sc_attitude_err=0x00000008
    #direct_signal_in_dd= 0x00008000
    #low_confidence_gps_eirp_estimate=0x00010000
    #rfi_detected= 0x00020000
    #sp_non_existent_error=0x00400000
    #ant_data_lut_range_error=0x01000000
    #anomalous_sampling_period=0x20000000
    #invalid_roll_state=0x40000000
    QF_out=(QF1&0x00000400>0)&(QF1&(0x00000002+0x00000008+0x00008000+0x00010000+
                                    0x00020000+0x00400000+0x01000000+0x20000000+0x40000000)==0)
    # incorrect_ddmi_antenna_selection=  0x00000001
    # sp_in_sidelobe           =       0x00000008
    # fatal_nst_outage        =      0x00000040
    # low_zenith_ant_gain     =     0x00000080
    QF_out2=(QF2&(0x00000001+0x00000008+0x00000040+0x00000080)==0)
    mask=QF_out&QF_out2
    df=df[mask]
    df['cygnss_satnum']=d['spacecraft_num'][:].data
    return df

In [3]:
fname="/loca/path/of/CyGNSSv3.0/netcdf_file.nc"

In [4]:
df=CYGdf(fname)

In [5]:
df

,sp_lat,sp_lon,sp_alt,prn_code,ddm_snr,fresnel_coeff,sp_inc_angle,sp_rx_gain,gps_eirp,gps_ant_gain_db_i,...,ddm_noise_floor,quality_flags,quality_flags_2,rx_to_sp_range,tx_to_sp_range,add_range_to_sp,track_id,sv_num,ddm_timestamp_utc,cygnss_satnum
0,23.986515,109.634422,-20.836042,28,5.817909,0.678631,37.706470,10.190885,758.676086,15.866661,...,7141.650879,269313,0,657122,21722946,2774.971436,1,44,2020-08-17 00:00:00.499261,5
2,23.504232,106.275970,-29.039190,17,0.586083,0.664454,49.440651,1.386414,1059.262939,16.607103,...,7213.754395,269313,0,778559,22258391,2201.358887,3,53,2020-08-17 00:00:00.499261,5
4,24.000086,109.660095,-20.763079,28,10.488630,0.678637,37.696133,10.196931,758.727539,15.865798,...,7091.507812,269313,0,657045,21722363,2775.432617,1,44,2020-08-17 00:00:00.999261,5
6,23.519461,106.300438,-29.033829,17,0.505437,0.664429,49.452473,1.364387,1059.650879,16.607737,...,7206.540039,269313,0,778720,22259254,2200.723389,3,53,2020-08-17 00:00:00.999261,5
10,23.534683,106.324928,-29.027788,17,0.704749,0.664405,49.464321,1.337728,1060.168213,16.608370,...,7400.208984,793601,0,778878,22260120,2200.090820,3,53,2020-08-17 00:00:01.499261,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688047,36.502045,140.068817,40.727253,28,-9999.000000,0.664556,44.743450,-9999.000000,931.371216,16.296696,...,5023.049805,7185,0,724171,22151086,2453.086182,1028,44,2020-08-17 23:53:22.999261,5
688051,36.508526,140.100891,40.589024,28,-9999.000000,0.664531,44.758839,-9999.000000,931.518860,16.297318,...,5031.399902,7185,0,724338,22152083,2452.319336,1028,44,2020-08-17 23:53:23.499261,5
688055,36.515007,140.132965,40.452248,28,-9999.000000,0.664507,44.774231,-9999.000000,931.606567,16.297937,...,5025.399902,7185,0,724508,22153079,2451.552246,1028,44,2020-08-17 23:53:23.999261,5
688059,36.521446,140.165070,40.316547,28,-9999.000000,0.664483,44.789677,-9999.000000,931.776917,16.298559,...,5041.350098,7185,0,724674,22154080,2450.774658,1028,44,2020-08-17 23:53:24.499261,5
